In [1]:
import pandas as pd

# Load tweets csv file
tweets = pd.read_csv('data/SE3B2D~1.csv')

In [9]:
# List fields
tweets.columns.values.tolist()

['id',
 'tweetId',
 'createdAt',
 'mention',
 'start',
 'end',
 'entity',
 'wikidata',
 'longitude',
 'latitude',
 'altitude',
 'osm',
 'geonames',
 'text']

In [5]:
# Select min and max dates
min_date = tweets['createdAt'].min()
max_date = tweets['createdAt'].max()
print('Min date: ', min_date)
print('Max date: ', max_date)

Min date:  2019-11-11T09:23:24Z
Max date:  2019-11-12T10:52:09Z
